In [1]:
import rdflib
from rdflib import URIRef, RDF, OWL, RDFS
from owlready2 import *
from sklearn.model_selection import train_test_split
from src.utils import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [2]:
dataset_name = 'OWL2DL-1'

**Step 1: Load ontology**

In [3]:
ontology = get_ontology(f'datasets/{dataset_name}.owl').load()

In [4]:
g = rdflib.Graph()
g.parse(f'datasets/{dataset_name}.owl') 
print(f'# Triples: {len(g)}')

# Triples: 55215


In [5]:
g_tbox = rdflib.Graph()
g_tbox.parse(f'datasets/{dataset_name}_TBOX.owl')  
print(f'# Triples: {len(g_tbox)}')

# Triples: 1135


**Step 2: Get all subject-resources (individuals)**

In [34]:
subject_resources = list(ontology.individuals())
print(f'# Subject-Resources: {len(subject_resources)}')

# Subject-Resources: 3668


In [35]:
subject_resources = subject_resources[100:103]

In [36]:
subject_resources

[OWL2Bench.U0C2D0L4, OWL2Bench.U0C4D0AssocP0, OWL2Bench.U0WC0D0AP3]

**Step 3: Generate g = description graph for each r**

In [37]:
for i, subject in enumerate(subject_resources):
    describe_graph = copy_graph(g_tbox)
    for triple in g.triples((None, None, URIRef(subject.iri))):
        describe_graph.add(triple) 
    for triple in g.triples((URIRef(subject.iri), None, None)):
        describe_graph.add(triple) 
    describe_graph.serialize(destination= f'g_{i}.owl', format="xml")

**Step 4: Generate i = inference graph for each graph g**

In [38]:
from owlready2 import *

In [51]:
onto = get_ontology('g_2.owl').load()

In [52]:
list(onto.individuals())

[OWL2Bench.U0WC0D0AP3]

In [53]:
with onto:
    sync_reasoner_pellet()

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\jena-core-2.10.0.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\jena-iri-0.9.5.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\jena-tdb-0.10.0.jar;C:\Users\julie\anaconda3\lib\site-packages\owlready2\pellet\jgr

KeyboardInterrupt: 